# **Testing Model**

## Cek hasil serving

In [1]:
import requests
from pprint import PrettyPrinter

pp = PrettyPrinter()
pp.pprint(requests.get("http://localhost:8080/v1/models/diabetes-prediction").json())

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1763528202'}]}


## Import Library

In [2]:
import base64, requests 
import pandas as pd 
import numpy as np
import tensorflow as tf

## Set Variable

In [3]:
# Endpoint REST TF Serving (host 8080 -> container 8501)
SERVING_URL = "http://localhost:8080/v1/models/diabetes-prediction:predict"

# Tipe fitur RAW sesuai schema (bukan _xf)
INT_FEATURES   = {"Pregnancies","Glucose","BloodPressure","SkinThickness","Insulin","Age"}
FLOAT_FEATURES = {"BMI","DiabetesPedigreeFunction"}
LABEL = "Outcome"

## Fungsi Helper

In [4]:
def tfexample_b64_from_dict(sample: dict) -> str:
    """Serialize dictionary ke tf.Example -> base64 string"""
    def _i(v): return tf.train.Feature(int64_list=tf.train.Int64List(value=[int(v)]))
    def _f(v): return tf.train.Feature(float_list=tf.train.FloatList(value=[float(v)]))
    feats = {}
    for k, v in sample.items():
        if k in INT_FEATURES:   feats[k] = _i(v)
        elif k in FLOAT_FEATURES: feats[k] = _f(v)
        else: raise ValueError(f"Fitur '{k}' tidak dikenali")
    ex = tf.train.Example(features=tf.train.Features(feature=feats))
    return base64.b64encode(ex.SerializeToString()).decode("utf-8")

def predict_one(sample: dict, threshold: float = 0.5) -> dict:
    """Kirim satu sampel ke endpoint TF Serving"""
    payload = {"instances": [{"b64": tfexample_b64_from_dict(sample)}]}
    r = requests.post(SERVING_URL, json=payload, timeout=30)
    r.raise_for_status()
    prob = float(r.json()["predictions"][0][0])
    return {"probability": prob, "prediction": int(prob >= threshold)}

## **Testing Inference Model**

In [5]:
# Testing Inference Model berdasarkan data dari dataset
# load dataset
try:
    df = pd.read_csv("data/diabetes.csv")
except FileNotFoundError:
    df = pd.read_csv("diabetes.csv")

# pastikan urutan kolom konsisten
cols = ["Pregnancies","Glucose","BloodPressure","SkinThickness","Insulin",
        "BMI","DiabetesPedigreeFunction","Age","Outcome"]
df = df[cols].copy()

# ambil 1 sampel uji berdasarkan index
idx = 2
row = df.iloc[idx]
sample_dict = row.drop(LABEL).to_dict()
print("Sample (RAW) dari dataset baris ke", idx)
for k in sample_dict: print(f"- {k}: {sample_dict[k]}")
print("Label ground-truth:", int(row[LABEL]))

# kirim ke TF Serving (gunakan helper tfexample_b64_from_dict)
payload = {"instances": [{"b64": tfexample_b64_from_dict(sample_dict)}]}
resp = requests.post(SERVING_URL, json=payload, timeout=30)
resp.raise_for_status()

prob = float(resp.json()["predictions"][0][0])
pred = int(prob >= 0.5)

print("\nHASIL PREDIKSI:")
print("Probabilitas (positif diabetes):", round(prob, 4))
print("Prediksi (1 = positif, 0 = negatif):", pred)


Sample (RAW) dari dataset baris ke 2
- Pregnancies: 8.0
- Glucose: 183.0
- BloodPressure: 64.0
- SkinThickness: 0.0
- Insulin: 0.0
- BMI: 23.3
- DiabetesPedigreeFunction: 0.672
- Age: 32.0
Label ground-truth: 1

HASIL PREDIKSI:
Probabilitas (positif diabetes): 0.7557
Prediksi (1 = positif, 0 = negatif): 1


In [8]:
# ambil beberapa sampel dari dataset
subset = df.sample(5)

# konversi ke serialized tf.Example base64 (gunakan helper tfexample_b64_from_dict)
instances = [{"b64": tfexample_b64_from_dict(r.drop(LABEL).to_dict())} for _, r in subset.iterrows()]

# kirim ke model
payload = {"instances": instances}
resp = requests.post(SERVING_URL, json=payload, timeout=30)
resp.raise_for_status()

# tampilkan hasil batch
preds = np.array(resp.json()["predictions"]).flatten()
results = pd.DataFrame({
    "Prob_Positive": preds,
    "Predicted_Label": (preds >= 0.5).astype(int),
    "True_Label": subset[LABEL].values
})
print(results)


   Prob_Positive  Predicted_Label  True_Label
0       0.349881                0           0
1       0.810560                1           1
2       0.021404                0           0
3       0.057467                0           0
4       0.924818                1           1


## **Inference Model (Input New Data)**

In [10]:
def input_interaktif():
    '''Input data baru secara interaktif dan prediksi'''
    print("Masukkan data:")
    s = {}
    s["Pregnancies"] = int(input("Pregnancies (int): "))
    s["Glucose"] = int(input("Glucose (int): "))
    s["BloodPressure"] = int(input("BloodPressure (int): "))
    s["SkinThickness"] = int(input("SkinThickness (int): "))
    s["Insulin"] = int(input("Insulin (int): "))
    s["BMI"] = float(input("BMI (float): "))
    s["DiabetesPedigreeFunction"] = float(input("DiabetesPedigreeFunction (float): "))
    s["Age"] = int(input("Age (int): "))

    print("\nData input:", s)
    res = predict_one(s, threshold=0.5)
    print("\nHASIL PREDIKSI:")
    print(f"Probabilitas positif diabetes: {res['probability']:.4f}")
    print(f"Prediksi: {res['prediction']} (1=positif, 0=negatif)")


In [11]:
# jalankan fungsi untuk uji dengan input manual (pasien negatif)
input_interaktif()

Masukkan data:

Data input: {'Pregnancies': 1, 'Glucose': 95, 'BloodPressure': 70, 'SkinThickness': 25, 'Insulin': 80, 'BMI': 23.5, 'DiabetesPedigreeFunction': 0.25, 'Age': 29}

HASIL PREDIKSI:
Probabilitas positif diabetes: 0.0224
Prediksi: 0 (1=positif, 0=negatif)


**Pasien Non-Diabetes (Outcome = 0)**  
Karakteristik umum:
- Pregnancies: 1
- Glucose: 95 → kadar gula darah normal (< 140 mg/dL)
- BloodPressure: 70 → tekanan darah normal (sekitar 120/80 mmHg)
- SkinThickness: 25 → ketebalan kulit wajar
- Insulin: 80 → kadar insulin dalam batas normal
- BMI: 23.5 → berat badan ideal (18.5–24.9)
- DiabetesPedigreeFunction: 0.25 → faktor genetik rendah
- Age: 29 → usia relatif muda

**Interpretasi:** Kemungkinan besar tidak diabetes karena kadar glukosa dan BMI masih dalam batas normal.

In [12]:
# jalankan fungsi untuk uji dengan input manual (pasien positif)
input_interaktif()

Masukkan data:

Data input: {'Pregnancies': 6, 'Glucose': 165, 'BloodPressure': 82, 'SkinThickness': 35, 'Insulin': 130, 'BMI': 35.2, 'DiabetesPedigreeFunction': 0.78, 'Age': 50}

HASIL PREDIKSI:
Probabilitas positif diabetes: 0.7929
Prediksi: 1 (1=positif, 0=negatif)


**Pasien Diabetes (Outcome = 1)**  
Karakteristik umum:
- Pregnancies: 6
- Glucose: 165 → kadar glukosa tinggi (≥ 140 mg/dL indikatif diabetes)
- BloodPressure: 82 → sedikit di atas normal
- SkinThickness: 35 → cenderung tinggi (indikasi lemak subkutan lebih banyak)
- Insulin: 130 → kadar insulin tinggi
- BMI: 35.2 → obesitas (BMI ≥ 30)
- DiabetesPedigreeFunction: 0.78 → faktor genetik sedang hingga tinggi
- Age: 50 → termasuk kategori risiko tinggi

**Interpretasi:** Kombinasi glukosa tinggi, obesitas, dan usia lanjut mengindikasikan diabetes.